In [29]:
import numpy as np
import plotly.graph_objects as go
from numpy.linalg import eigh
from math import pi

def H_lieb(kx, ky, a=0.0):
    return np.array([
        [a, 2*np.cos(kx), 2*np.cos(ky)],
        [2*np.cos(kx), 0, 0],
        [2*np.cos(ky), 0, 0]
    ], dtype=float)

nk = 101
kx_vals = np.linspace(0, pi, nk)
ky_vals = np.linspace(0, pi, nk)
KX, KY = np.meshgrid(kx_vals, ky_vals)

def compute_bands(a):
    E = np.zeros((3, nk, nk))
    for i, kx in enumerate(kx_vals):
        for j, ky in enumerate(ky_vals):
            E[:, j, i] = np.sort(eigh(H_lieb(kx, ky, a=a))[0])
    return E

#parameters
a = 0.0
E = compute_bands(a)

fig = go.Figure()

colors = ['reds', 'greens', 'blues']
for n in range(3):
    fig.add_surface(
        x=KX, y=KY, z=E[n],
        colorscale=colors[n],
        showscale=False,
        opacity=0.9
    )

fig.update_layout(
    title=f"Lieb lattice band structure (a = {a})",
    scene=dict(
        xaxis_title="kx",
        yaxis_title="ky",
        zaxis_title="Energy",
        zaxis=dict(range=[-3, 3])
    ),
    width=700, height=600
)
fig.show()


In [28]:
import numpy as np
import plotly.graph_objects as go
from numpy.linalg import eigh
from math import pi

def H_lieb(kx, ky, a=0.0):
    return np.array([
        [a, 2*np.cos(kx), 2*np.cos(ky)],
        [2*np.cos(kx), 0, 0],
        [2*np.cos(ky), 0, 0]
    ], dtype=float)

nk = 101
kx_vals = np.linspace(0, pi, nk)
ky_vals = np.linspace(0, pi, nk)
KX, KY = np.meshgrid(kx_vals, ky_vals)

def compute_bands(a):
    E = np.zeros((3, nk, nk))
    for i, kx in enumerate(kx_vals):
        for j, ky in enumerate(ky_vals):
            E[:, j, i] = np.sort(eigh(H_lieb(kx, ky, a=a))[0])
    return E

#parameters
a = 0.1
E = compute_bands(a)

fig = go.Figure()

colors = ['reds', 'greens', 'blues']
for n in range(3):
    fig.add_surface(
        x=KX, y=KY, z=E[n],
        colorscale=colors[n],
        showscale=False,
        opacity=0.9
    )

fig.update_layout(
    title=f"Lieb lattice band structure (a = {a})",
    scene=dict(
        xaxis_title="kx",
        yaxis_title="ky",
        zaxis_title="Energy",
        zaxis=dict(range=[-3, 3])
    ),
    width=700, height=600
)
fig.show()

In [30]:
import numpy as np
import plotly.graph_objects as go
from numpy.linalg import eigh
from math import pi

def H_lieb(kx, ky, a=0.0):
    return np.array([
        [a, 2*np.cos(kx), 2*np.cos(ky)],
        [2*np.cos(kx), 0, 0],
        [2*np.cos(ky), 0, 0]
    ], dtype=float)

nk = 101
kx_vals = np.linspace(0, pi, nk)
ky_vals = np.linspace(0, pi, nk)
KX, KY = np.meshgrid(kx_vals, ky_vals)

def compute_bands(a):
    E = np.zeros((3, nk, nk))
    for i, kx in enumerate(kx_vals):
        for j, ky in enumerate(ky_vals):
            E[:, j, i] = np.sort(eigh(H_lieb(kx, ky, a=a))[0])
    return E

#parameters
a = -0.1
E = compute_bands(a)

fig = go.Figure()

colors = ['reds', 'greens', 'blues']
for n in range(3):
    fig.add_surface(
        x=KX, y=KY, z=E[n],
        colorscale=colors[n],
        showscale=False,
        opacity=0.9
    )

fig.update_layout(
    title=f"Lieb lattice band structure (a = {a})",
    scene=dict(
        xaxis_title="kx",
        yaxis_title="ky",
        zaxis_title="Energy",
        zaxis=dict(range=[-3, 3])
    ),
    width=700, height=600
)
fig.show()